# Scrapper cédula profesional

## Conexión al navegador

In [ ]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://cedulaprofesionalsep.online/#Consulta_de_Cedula_Profesional'
driver.get(start_url)
# Modificar el tamaño de la ventana
#driver.execute_script("document.body.style.zoom='50%'")

> Función para obtener el tamaño de una solicitud HTTP

In [30]:
def get_request_size(url):
    response = requests.get(url)
    return len(response.content)

## Leer archivo CSV y obtener lista de nombres

In [7]:
init=2
final=500

df = pd.read_excel('Z:/Data/HSBC/Asignaciones/ASG_TDC_IA_ENERO_JUNIO.xlsx')
df['Nombre'] = df.Nombre.apply(lambda X: X.replace('/', ' ').replace('*', '').rstrip())
df_filtered = df['Nombre']
df_filtered.drop_duplicates(inplace=True)
df_input = pd.DataFrame(df_filtered.reset_index(drop=True)).iloc[init:final]



df_input['NombreSplit'] = df_input.Nombre.apply(lambda X: split_name(X))
df_input['Nombre'] = df_input.NombreSplit.apply(lambda X: X[0])
df_input['Paterno'] = df_input.NombreSplit.apply(lambda X: X[1])
df_input['Materno'] = df_input.NombreSplit.apply(lambda X: X[2])

# Concatenación
df_input['Nombre_Completo'] = df_input.apply(lambda row: ' '.join([row['Nombre'], row['Paterno'], row['Materno']]), axis = 1)
# Cambiar nombre
#df.rename(columns={'Nombre':'Nombre'}, inplace=True)
df_input.tail()

TypeError: sequence item 2: expected str instance, NoneType found

In [4]:
df_input.head()

,Nombre,NombreSplit,Paterno,Materno,Nombre_Completo
192,jose de jesus,"[jose de jesus, hernandez, hernandez]",hernandez,hernandez,jose de jesus hernandez hernandez
193,jose adan,"[jose adan, perez, tamayo]",perez,tamayo,jose adan perez tamayo
194,alejandro,"[alejandro, perez, alcantara]",perez,alcantara,alejandro perez alcantara
195,j ascencion,"[j ascencion, pina, patino]",pina,patino,j ascencion pina patino
196,miguel angel,"[miguel angel, sanchez, ramos]",sanchez,ramos,miguel angel sanchez ramos


## Scrapper

In [5]:
# Código para modificar la ventana
driver.execute_script("window.scrollTo(10, 7600);")

In [ ]:
data=[]
#total_data_downloaded = 0
for index, row in df_input.iterrows():
    nombre_persona = row['Nombre_Completo']
   
    # Se escrolea hasta la parte específica 
    driver.execute_script("window.scrollTo(10, 7600);")
    time.sleep(5)
    
    # Se ingresan las variables del scrapeo
    nombre=row['Nombre']
    paterno=row['Paterno']
    materno=row['Materno']
    
    # Ingresar nombres
    celda_nombre = driver.find_element(By.XPATH, '//*[@id="input-nombre"]')
    celda_nombre.clear()
    celda_nombre.send_keys(str(nombre))

    #Ingresar Apellido Paterno
    celda_paterno = driver.find_element(By.XPATH, '//*[@id="input-apaterno"]')
    celda_paterno.clear()
    celda_paterno.send_keys(str(paterno))

    # Ingresar Apellido Materno
    celda_materno = driver.find_element(By.XPATH, '//*[@id="input-amaterno"]')
    celda_materno.clear()
    celda_materno.send_keys(str(materno))

    # Click en botón buscar
    buscar = driver.find_element(By.XPATH, '//*[@id="container-form-1"]/form/div[4]/div/input')
    buscar.click()
    time.sleep(2)

    max_wait_time = 5  # Tiempo máximo de espera en segundos
    start_time = time.time()  # Momento de inicio de espera

    while time.time() - start_time < max_wait_time:
        try:
            alert = driver.switch_to.alert
            alert.accept()
            driver.refresh()
            break  # Salir del bucle si se encuentra la ventana emergente
        except:

            time.sleep(1)  # Esperar 1 segundo antes de intentar nuevamente
            
            


    time.sleep(5)
    
    # Esperar a que los elementos estén visibles
    ver_botones = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'ver-mas')))

    
    for ver in ver_botones:
        nombre_boton = ver.get_attribute("value")
        print(nombre_boton)
        if unidecode(nombre_boton.lower()) == unidecode(nombre_persona.lower()):
            cantidad_registros = len(ver_botones)

            ver.click()
            
            # Encontrar los elementos de entrada de texto por su id
            input_cedula = driver.find_element(By.ID, "input-cedula-result")
            input_tipo = driver.find_element(By.ID, "input-tipo-result")
            input_sexo = driver.find_element(By.ID, "input-sexo-result")
            input_nombre = driver.find_element(By.ID, "input-nombre-result")
            input_paterno = driver.find_element(By.ID, "input-paterno-result")
            input_materno = driver.find_element(By.ID, "input-materno-result")
            input_escuela = driver.find_element(By.ID, "input-escuela-result")
            input_titulo = driver.find_element(By.ID, "input-titulo-result")

            # Obtener los valores de los campos
            valor_cedula = input_cedula.get_attribute("value")
            valor_tipo = input_tipo.get_attribute("value")
            valor_sexo = input_sexo.get_attribute("value")
            valor_nombre = input_nombre.get_attribute("value")
            valor_paterno = input_paterno.get_attribute("value")
            valor_materno = input_materno.get_attribute("value")
            valor_escuela = input_escuela.get_attribute("value")
            valor_titulo = input_titulo.get_attribute("value")

            # Agregar los valores a la lista de diccionarios
            data.append({
            "Cédula": valor_cedula,
            "Tipo de Cédula": valor_tipo,
            "Sexo": valor_sexo,
            "Nombre": valor_nombre,
            "Paterno": valor_paterno,
            "Materno": valor_materno,
            "Escuela": valor_escuela,
            "Título": valor_titulo
            })
            # Encontrar los elementos de entrada de texto por su id
            cerrar = driver.find_element(By.XPATH, '//*[@id="cerrar_resut_personal"]')
            cerrar.click()
       
        
        # Obtener el tamaño de la página descargada
        #page_source_size = len(driver.page_source.encode('utf-8'))
        #total_data_downloaded += page_source_size
        
driver.quit()            
df_data = pd.DataFrame(data)
df_data.to_csv('Z:/Data/HSBC/SCRAPPING/Cédulas/0_500_.csv')
    

                



In [ ]:
df_data.tail(10)

In [37]:
nombre_buscado = "jose de jesus"
resultado = df_input.loc[df_input['Nombre'] == nombre_buscado]
resultado

,Nombre,NombreSplit,Paterno,Materno
192,jose de jesus,"[jose de jesus, hernandez, hernandez]",hernandez,hernandez


In [39]:
# Convertir los datos en un DataFrame y guardar en CSV              
df_data = pd.DataFrame(data)
df_data.to_csv('Z:/Data/HSBC/SCRAPPING/Cédulas/0_192_.csv')
    

# Scraper marcatel

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from datetime import datetime, timedelta, date
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import json
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import warnings
warnings.filterwarnings('ignore')

def marcatel_automation():
    url = "https://tink.marcatel.com.mx/Login.aspx"
    
    load_dotenv(find_dotenv()) # Load the .env file.
    userid = os.getenv("USER_MARCATEL")
    passwo = os.getenv("PASSWORD_MARCATEL")
    
   
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    user = wait.until(EC.presence_of_element_located((By.ID, "usuario")))
    user.click()
    user.send_keys(userid)
    contra = wait.until(EC.presence_of_element_located((By.ID, "contrasena")))
    
        # Pasar credenciales
    contra.click()
    contra.send_keys(passwo)
    init_button = wait.until(EC.element_to_be_clickable((By.ID, "btnini")))
    init_button.click()
    time.sleep(10)
        # Abrir ventana de reportes SMS
        # Redirecciona a la URL deseada
    driver.get("https://tink.marcatel.com.mx/Reportes/wfReporteRespuestas.aspx")
        # Espera unos segundos antes de cerrar el navegador
    time.sleep(10)
    
    # Get current date and time
    current_date = datetime.now()
    
    # Yesterday's date
    yersterday_date = current_date - timedelta(days=30)
    
    # Format date
    formatted_date = yersterday_date.strftime("%Y-%m-%d")
    print(formatted_date)
    # Date to marcatel
    date_marcatel = wait.until(EC.presence_of_element_located((By.ID, 'start')))
    date_marcatel.click()
    date_marcatel.clear()
    date_marcatel.send_keys(formatted_date)
    
    time.sleep(6)

    reporte = wait.until(EC.element_to_be_clickable((By.ID, "generarReporte")))
    reporte.click()
    time.sleep(30)
    exportar = driver.find_element(By.ID, "excel")
    driver.execute_script("arguments[0].scrollIntoView();", exportar)
    exportar.click()
    time.sleep(10)
    driver.quit()
marcatel_automation()


2023-09-10


: 

# Scraper Partidos Políticos y gobernadores

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import os
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')


def gobernadores_scraper():
    url = 'https://www.conago.org.mx/gobernadores'
    

   
    # Configuración para evitar notificaciones
    chrome_options = Options()
    #options.add_argument("window-size=1200x600")
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    #"profile.default_content_settings.popups":0,
    "download.default_directory":"/opt/airflow/outputs/Gobernadores/"})
    # Configuración para ingresar al explorador

    remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome( options=chrome_options)
    driver.get(url)

    # Localiza los elementos de interés
    elementos = driver.find_elements(By.CLASS_NAME, 'col-xs-12.col-md-8.texto.rcGobernadores')


    # Crear o abrir el archivo CSV
    with open('datos.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Nombre', 'Fecha de Inicio', 'Fecha de Fin', 'Estado']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()  # Escribir la fila de encabezados
        
        for _ in range(len(elementos)):
            # Obtén el elemento específico y el estado
            elemento = elementos[_]
            estado_element = elemento.find_element(By.CLASS_NAME, 'textMedio')
            estado = estado_element.text
            estado_element.click()  # Hacer clic en el enlace
            
            # Obtener los elementos de interés en la página de detalle del estado
            elementos_estado = driver.find_elements(By.CLASS_NAME, 'col-xs-12.col-sm-6.col-md-4.pb15')

            for elemento_estado in elementos_estado:
                nombre_gobernador = elemento_estado.find_element(By.TAG_NAME, 'h4').text
                html = elemento_estado.get_attribute('innerHTML')
                
                # Usar expresión regular para encontrar las fechas
                fechas = re.search(r'(\d{2}/\d{2}/\d{4}) a (\d{2}/\d{2}/\d{4})', html)
                
                if fechas:
                    fecha_inicio = fechas.group(1)
                    fecha_fin = fechas.group(2)
                else:
                    fecha_inicio = "Fecha no disponible"
                    fecha_fin = "Fecha no disponible"
                
                # Imprimir para verificar
                print(nombre_gobernador, fecha_inicio, fecha_fin)
                
                # Escribir los datos en el archivo CSV
                writer.writerow({'Nombre': nombre_gobernador, 'Fecha de Inicio': fecha_inicio,
                                'Fecha de Fin': fecha_fin, 'Estado': estado})

            driver.back()  # Regresar a la página anterior

        # Cerrar el navegador
        driver.quit()
gobernadores_scraper()



def partidos_scraper():
    # Configura el controlador de Chrome
    driver = webdriver.Chrome()

    # Abre la página web de Wikipedia
    url = 'https://es.wikipedia.org/wiki/Wikipedia:Portada'
    driver.get(url)

    # Leer los nombres de los gobernadores desde el archivo CSV
    nombres_gobernadores = []
    resultados = []

    with open('datos.csv', 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            nombre_completo = row['Nombre']
            
            # Buscar el último punto en el nombre
            ultimo_punto_index = nombre_completo.rfind('.')
            if ultimo_punto_index != -1:
                nombre_completo = nombre_completo[ultimo_punto_index+1:].strip()
            
            nombres_gobernadores.append(nombre_completo)

    # ... (código anterior) ...

    # Iterar sobre los nombres y buscar en la página web
    for nombre in nombres_gobernadores:
        buscar_gobernador = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cdx-text-input__input'))
        )

        # Usar ActionChains para borrar y escribir
        actions = ActionChains(driver)
        actions.click(buscar_gobernador).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).send_keys(nombre).perform()
        buscar_gobernador = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cdx-text-input__input'))
        )
        buscar_gobernador.submit()
        buscar_gobernador = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cdx-text-input__input'))
        )
        # Esperar a que se cargue la página del artículo
        driver.implicitly_wait(10)

        # Verificar si la página muestra resultados de búsqueda
        try:
            no_results_heading = driver.find_element(By.CLASS_NAME, 'firstHeading.mw-first-heading')
            if no_results_heading.text == 'Resultados de la búsqueda':
                print(f"No se encontró información para {nombre}")
                print("---")
                continue
        except NoSuchElementException:
            pass

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        infobox = soup.find('table', {'class': 'infobox biography vcard'})

        if infobox:
            partido_element = infobox.find('th', string='Partido político')
            if partido_element:
                partido_td = partido_element.find_next('td')
                partido = partido_td.get_text(strip=True)

                # Buscar el año dentro de los paréntesis
                anio_inicio_element = partido_td.find('span', {'style': 'font-size:85%;'})
                if anio_inicio_element:
                    anio_inicio = anio_inicio_element.get_text(strip=True).strip('()')
                else:
                    anio_inicio = "No disponible"

                resultados.append({"Nombre": nombre, "Partido político": partido, "Año de inicio": anio_inicio})
            else:
                resultados.append({"Nombre": nombre, "Partido político": "No disponible", "Año de inicio": "No disponible"})
        else:
            resultados.append({"Nombre": nombre, "Partido político": "No disponible", "Año de inicio": "No disponible"})

    # Guardar los resultados en un archivo CSV
    with open('resultados_gobernadores.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Nombre", "Partido político", "Año de inicio"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(resultados)

    # Cerrar el navegador
    driver.quit()
partidos_scraper()

Mtra. María Teresa Jiménez Esquivel 01/10/2022 30/09/2028
C.P. Martín Orozco Sandoval 01/12/2016 30/09/2022
Ing. Carlos Lozano de la Torre 01/12/2010 30/11/2016
Ing. Luis Armando Reynoso Femat 01/12/2004 30/11/2010
Lic. Juan José León Rubio 24/08/2004 30/11/2004
Sr. Felipe González González 01/12/1998 23/08/2004
Mtra. Marina del Pilar Ávila Olmeda 01/11/2021 31/10/2027
Ing. Jaime Bonilla Valdez 01/11/2019 31/10/2021
Lic. Francisco Arturo Vega de Lamadrid 01/11/2013 31/10/2019
Lic. José Guadalupe Osuna Millán 01/11/2007 31/10/2013
Lic. Eugenio Elorduy Walther 01/11/2001 31/10/2007
Mtro. Víctor Manuel Castro Cosío 10/09/2021 09/09/2027
Mtro. Carlos Mendoza Davis 10/09/2015 09/09/2021
C. Marcos Alberto Covarrubias Villaseñor 05/04/2011 09/09/2015
Ing. Narciso Agúndez Montaño 05/04/2005 04/04/2011
Lic. Leonel Efraín Cota Montaño 05/04/1999 04/04/2005
Mtra. Layda Elena Sansores San Román 16/09/2021 15/09/2027
Lic. Carlos Miguel Aysa González 13/06/2019 15/09/2021
Lic. Rafael Alejandro Moren

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=118.0.5993.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6730A8EF2+54786]
	(No symbol) [0x00007FF673015612]
	(No symbol) [0x00007FF672ECA64B]
	(No symbol) [0x00007FF672ECF116]
	(No symbol) [0x00007FF672ED14AA]
	(No symbol) [0x00007FF672F45394]
	(No symbol) [0x00007FF672F2BE6A]
	(No symbol) [0x00007FF672F44D02]
	(No symbol) [0x00007FF672F2BC43]
	(No symbol) [0x00007FF672F00941]
	(No symbol) [0x00007FF672F01B84]
	GetHandleVerifier [0x00007FF6733F7F52+3524194]
	GetHandleVerifier [0x00007FF67344D800+3874576]
	GetHandleVerifier [0x00007FF673445D7F+3843215]
	GetHandleVerifier [0x00007FF673145086+694166]
	(No symbol) [0x00007FF673020A88]
	(No symbol) [0x00007FF67301CA94]
	(No symbol) [0x00007FF67301CBC2]
	(No symbol) [0x00007FF673

In [5]:
df_gob = pd.read_csv("datos.csv")
df_gob

,Nombre,Fecha de Inicio,Fecha de Fin,Estado
0,Mtra. María Teresa Jiménez Esquivel,01/10/2022,30/09/2028,Aguascalientes
1,C.P. Martín Orozco Sandoval,01/12/2016,30/09/2022,Aguascalientes
2,Ing. Carlos Lozano de la Torre,01/12/2010,30/11/2016,Aguascalientes
3,Ing. Luis Armando Reynoso Femat,01/12/2004,30/11/2010,Aguascalientes
4,Lic. Juan José León Rubio,24/08/2004,30/11/2004,Aguascalientes
...,...,...,...,...
170,Lic. David Monreal Ávila,12/09/2021,11/09/2027,Zacatecas
171,L.C. Alejandro Tello Cristerna,12/09/2016,11/09/2021,Zacatecas
172,Lic. Miguel Alejandro Alonso Reyes,12/09/2010,11/09/2016,Zacatecas
173,Lic. Amalia D. García Medina,12/09/2004,11/09/2010,Zacatecas


In [6]:
df_pp = pd.read_csv("resultados_gobernadores.csv")

,Nombre,Partido político,Año de inicio
0,María Teresa Jiménez Esquivel,Partido Acción Nacional(desde 2002),desde 2002
1,Martín Orozco Sandoval,Partido Acción Nacional,No disponible
2,Carlos Lozano de la Torre,Partido Revolucionario Institucional,No disponible
3,Luis Armando Reynoso Femat,Fuerza por México(2021)[1]​Partido Acción Naci...,No disponible
4,Juan José León Rubio,Partido Acción Nacional,No disponible
...,...,...,...
140,Patricio José Patrón Laviada,Partido Acción Nacional,No disponible
141,David Monreal Ávila,Movimiento Regeneración Nacional(desde 2014)Pa...,desde 2014
142,Alejandro Tello Cristerna,Partido Revolucionario Institucional,No disponible
143,Miguel Alejandro Alonso Reyes,Partido Revolucionario Institucional(1993-1998...,"1993-1998, desde 2009"


In [11]:
filtrado_ejemplo = df_pp.loc[df_pp['Año de inicio']!='No disponible']
filtrado_ejemplo['Partido político']

0                    Partido Acción Nacional(desde 2002)
18     Movimiento Regeneración Nacional(desde 2014)Pa...
54     Movimiento Regeneración Nacional(desde 2017)In...
59     Sin Partido(desde 2012)[2]​[3]​Partido de la R...
61     Partido Acción Nacional(1992-2015; desde 2021)...
67     Partido Encuentro Social(2018)Partido Revoluci...
68     Partido Encuentro Social(2018)Partido Revoluci...
69     Movimiento Regeneración Nacional(desde 2017)Pa...
73     Fuerza por México(desde 2021)[1]​Partido Acció...
78     Partido Acción Nacional(desde 1998)Partido Rev...
79                      Movimiento Ciudadano(desde 2015)
80     Independiente(desde 2014)Partido Revolucionari...
81     Independiente(desde 2003)Partido Revolucionari...
82     Independiente(desde 2014)Partido Revolucionari...
88     Movimiento Ciudadano(desde 2011)Convergencia(2...
89     Independiente(desde 2021)Partido Revolucionari...
94                    Partido Acción Nacional(2009-2018)
105      Partido Revolucionario

# Scraper SHF 

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
import PyPDF2
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

def ift_scraper():
    chrome_options = Options()
    #chrome_options.add_argument('--headless')
    #chrome_options.add_argument('--disable-dev-shm-usage')   
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    })
    
    #remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome( options=chrome_options)
    # Scraping part
    url = "https://www.gob.mx/shf/documentos/indice-shf-de-precios-de-la-vivienda-en-mexico-2021-a-2025?state=published"
    # Configuración para ingresar al explorador
    #driver = webdriver.Chrome(options = chrome_options)
    driver.get(url)
    wait= WebDriverWait(driver, 10)
        
        # Encuentra todos los elementos li con la clase "clearfix documents"
    document_elements = driver.find_elements(By.CSS_SELECTOR, "li.clearfix.documents")
        
        # Verifica si hay al menos un elemento
    if document_elements:
            # Encuentra el primer elemento de la lista
        first_document = document_elements[0]
            
            # Encuentra botón de descarga
        pdf_link = first_document.find_element(By.CSS_SELECTOR, "a[href$='.pdf']")
            
            # Obtiene el enlace del atributo href
        pdf_url = pdf_link.get_attribute("href")
            
            # Abre el enlace en una nueva ventana
        driver.execute_script("window.open('" + pdf_url + "', '_blanck');")
            
            # Cambia el enfoque a la nueva ventana
        driver.switch_to.window(driver.window_handles[-1])
            
            # Obtener URL actual
        url_actual = driver.current_url
        print("URL actual", url_actual)
            
            # Ruta destino
        ruta_destino = "SHF.pdf"
            
            # Realiza la solicitud HTTP para descargar el archivo
        response=requests.get(url_actual)
            
            # Verifica si la descarga fue exitosa (código de estado 200)
        if response.status_code == 200:
                # Abre un archivo en modo binario y guarda el contenido del PDF
            with open(ruta_destino, 'wb') as pdf_file:
                pdf_file.write(response.content)
    driver.quit()


    def pdf_convert():
        pdf_file = "SHF.pdf"
            
        # Abre el archivo PDF
        pdf_file = open(pdf_file, 'rb')

        # Crea un objeto PdfFileReader para leer el PDF
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Inicializa una lista para almacenar el texto de cada página
        page_texts = []

        # Itera a través de las páginas y extrae el texto
        for page in pdf_reader.pages:
            page_texts.append(page.extract_text())

        # Cierra el archivo PDF
        pdf_file.close()

        # Combina el texto de todas las páginas en un solo texto
        all_text = '\n'.join(page_texts)

        # Divide el texto en líneas
        lines = all_text.split('\n')

        # Crea un DataFrame a partir de las líneas
        df = pd.DataFrame(lines, columns=['Texto'])

        ruta_csv = "SHF.csv"    
        df.to_csv(ruta_csv, index=False)
        return df
    pdf_convert()

    def extractor_tablas():
        df = pd.read_csv("SHF.csv")
        df = df.drop(df.index[48:])
        # Reajusta el índice para que comience desde 0
        df = df.reset_index(drop=True)
        # Elimina la primera fila, que contiene el encabezado, y guarda el resultado en un nuevo DataFrame
        df_sin_encabezado = df.iloc[1:].copy()

        # Extrae la columna 'Valores'
        df_sin_encabezado['Valores'] = df_sin_encabezado['Texto'].str.extract(r'(\d[\d. \s]+)')
        # Dividir la columna 'Valores' en múltiples columnas usando espacios en blanco como separadores
        columnas_valores = df_sin_encabezado['Valores'].str.split(expand=True)
        # Eliminar filas que contienen valores NaN (en este caso, la fila que contiene "Na")
        columnas_valores = columnas_valores.dropna()

        df_sin_encabezado['Estado'] = (df_sin_encabezado['Texto'].str.extract('([^0-9]+)'))
        # Combina la columna 'Estado' de df_sin_encabezado con columnas_valores
        columnas_valores.insert(0, 'Estado', df_sin_encabezado['Estado'].values)
        # Acceder a la segunda columna por posición
        segunda_columna = columnas_valores.iloc[:, 1]
        # Lista de índices de filas a modificar 
        filas_a_modificar = [0, 33, 34]

        for fila_idx in filas_a_modificar:
            # Modificar los valores de la segunda columna
            segunda_columna.iloc[fila_idx] = str(segunda_columna.iloc[fila_idx])[1:]

        columnas_valores = columnas_valores.rename(columns={columnas_valores.columns[0]: 'Estado'})


        #Obtener la longitud 
        num_columnas = len(columnas_valores.columns)
        # Crear una lista de años desde 2015 hasta 2024
        years = range(2015, 2025)
        # Crear una lista de números romanos del I al IV
        romanos = ['I', 'II', 'III', 'IV']
        # Crea una lista de nombres de columna basados en el patrón deseado
        nombres_columnas = ["Estado"] + [f"{anno}-{romano}" for anno in years for romano in romanos]
        # Asegurarse de que la lista de nombres de columna tenga la misma longitud que la tabla
        nombres_columnas = nombres_columnas[:num_columnas]
        columnas_valores.columns = nombres_columnas
        # Guardar el DataFrame modificado en un archivo CSV
        columnas_valores.to_csv("SHF_extract.csv", index=False)
    extractor_tablas()
ift_scraper()

URL actual https://www.gob.mx/cms/uploads/attachment/file/848146/_ndice_SHF_T2_2023.pdf


# Scraper Desempleo

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
import PyPDF2
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities



def INEGI_desempleo_scraper():
    url = "https://www.inegi.org.mx/app/tabulados/default.html?nc=624"
    
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    #"download.default_directory": "C:/Users/DSTHREE/Documents/GITHUB/airflow/dags/utils",
})
    # Configuración para ingresar al explorador
    #driver = webdriver.Chrome(options = chrome_options)
    remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait= WebDriverWait(driver, 10)
    button = wait.until(EC.element_to_be_clickable((By.ID, "aCsv")))
    button.click()
    time.sleep(20)
    driver.quit()
    
    def data_transformation():
        df = pd.read_csv("Tabulado.csv")
        # Encuentra los índices donde comienzan los años en la columna 'Periodo'
        indices_anios = df[df['Periodo'].str.isnumeric()].index
        nuevo_df = df.T.iloc[1:].reset_index(drop=True)
        nuevo_df = nuevo_df.dropna(axis =1)
        num_columnas = len(nuevo_df.columns)
            
        years = range(2020,2025)
        # Lista de números romanos
        romanos = ['I', 'II', 'III', 'IV']
        # Crea una lista de nombres de columnas basados en el patrón
        nombres_columnas = [f"{anno}-{romano}" for anno in years for romano in romanos]
        nombres_columnas = nombres_columnas[:num_columnas]
        #Asigna los nombres de las columnas al nuevo DataFrame
        nuevo_df.columns = nombres_columnas 
        # Agrega una columna 'Estado' al nuevo DataFrame
        nuevo_df.insert(0, 'Estado', df.columns[1:])

        # Restablece el índice
        nuevo_df.reset_index(drop=True, inplace=True)
        nuevo_df.to_csv("Tabulado.csv", index = False)
    data_transformation()
INEGI_desempleo_scraper()

# Unión df

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
import PyPDF2


def ift_scraper():
    chrome_options = Options()
    #chrome_options.add_argument('--headless')
    #chrome_options.add_argument('--disable-dev-shm-usage')   
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    })
    
    #remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome( options=chrome_options)
    # Scraping part
    url = "https://www.gob.mx/shf/documentos/indice-shf-de-precios-de-la-vivienda-en-mexico-2021-a-2025?state=published"
    # Configuración para ingresar al explorador
    #driver = webdriver.Chrome(options = chrome_options)
    driver.get(url)
    wait= WebDriverWait(driver, 10)
        
        # Encuentra todos los elementos li con la clase "clearfix documents"
    document_elements = driver.find_elements(By.CSS_SELECTOR, "li.clearfix.documents")
        
        # Verifica si hay al menos un elemento
    if document_elements:
            # Encuentra el primer elemento de la lista
        first_document = document_elements[0]
            
            # Encuentra botón de descarga
        pdf_link = first_document.find_element(By.CSS_SELECTOR, "a[href$='.pdf']")
            
            # Obtiene el enlace del atributo href
        pdf_url = pdf_link.get_attribute("href")
            
            # Abre el enlace en una nueva ventana
        driver.execute_script("window.open('" + pdf_url + "', '_blanck');")
            
            # Cambia el enfoque a la nueva ventana
        driver.switch_to.window(driver.window_handles[-1])
            
            # Obtener URL actual
        url_actual = driver.current_url
        print("URL actual", url_actual)
            
            # Ruta destino
        ruta_destino = "SHF.pdf"
            
            # Realiza la solicitud HTTP para descargar el archivo
        response=requests.get(url_actual)
            
            # Verifica si la descarga fue exitosa (código de estado 200)
        if response.status_code == 200:
                # Abre un archivo en modo binario y guarda el contenido del PDF
            with open(ruta_destino, 'wb') as pdf_file:
                pdf_file.write(response.content)
    driver.quit()


    def pdf_convert():
        pdf_file = "SHF.pdf"
            
        # Abre el archivo PDF
        pdf_file = open(pdf_file, 'rb')

        # Crea un objeto PdfFileReader para leer el PDF
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Inicializa una lista para almacenar el texto de cada página
        page_texts = []

        # Itera a través de las páginas y extrae el texto
        for page in pdf_reader.pages:
            page_texts.append(page.extract_text())

        # Cierra el archivo PDF
        pdf_file.close()

        # Combina el texto de todas las páginas en un solo texto
        all_text = '\n'.join(page_texts)

        # Divide el texto en líneas
        lines = all_text.split('\n')

        # Crea un DataFrame a partir de las líneas
        df = pd.DataFrame(lines, columns=['Texto'])

        ruta_csv = "SHF.csv"    
        df.to_csv(ruta_csv, index=False)
        return df
    pdf_convert()

    def extractor_tablas():
        df = pd.read_csv("SHF.csv")
        df = df.drop(df.index[48:])
        # Reajusta el índice para que comience desde 0
        df = df.reset_index(drop=True)
        # Elimina la primera fila, que contiene el encabezado, y guarda el resultado en un nuevo DataFrame
        df_sin_encabezado = df.iloc[1:].copy()

        # Extrae la columna 'Valores'
        df_sin_encabezado['Valores'] = df_sin_encabezado['Texto'].str.extract(r'(\d[\d. \s]+)')
        # Dividir la columna 'Valores' en múltiples columnas usando espacios en blanco como separadores
        columnas_valores = df_sin_encabezado['Valores'].str.split(expand=True)
        # Eliminar filas que contienen valores NaN (en este caso, la fila que contiene "Na")
        columnas_valores = columnas_valores.dropna()

        df_sin_encabezado['Estado'] = (df_sin_encabezado['Texto'].str.extract('([^0-9]+)'))
        # Combina la columna 'Estado' de df_sin_encabezado con columnas_valores
        columnas_valores.insert(0, 'Estado', df_sin_encabezado['Estado'].values)
        # Acceder a la segunda columna por posición
        segunda_columna = columnas_valores.iloc[:, 1]
        # Lista de índices de filas a modificar 
        filas_a_modificar = [0, 33, 34]

        for fila_idx in filas_a_modificar:
            # Modificar los valores de la segunda columna
            segunda_columna.iloc[fila_idx] = str(segunda_columna.iloc[fila_idx])[1:]

        columnas_valores = columnas_valores.rename(columns={columnas_valores.columns[0]: 'Estado'})


        #Obtener la longitud 
        num_columnas = len(columnas_valores.columns)
        # Crear una lista de años desde 2015 hasta 2024
        years = range(2015, 2025)
        # Crear una lista de números romanos del I al IV
        romanos = ['I', 'II', 'III', 'IV']
        # Crea una lista de nombres de columna basados en el patrón deseado
        nombres_columnas = ["Estado"] + [f"{anno}-{romano}" for anno in years for romano in romanos]
        # Asegurarse de que la lista de nombres de columna tenga la misma longitud que la tabla
        nombres_columnas = nombres_columnas[:num_columnas]
        columnas_valores.columns = nombres_columnas
        # Guardar el DataFrame modificado en un archivo CSV
        columnas_valores.to_csv("SHF_extract.csv", index=False)
    extractor_tablas()
ift_scraper()

def INEGI_desempleo_scraper():
    url = "https://www.inegi.org.mx/app/tabulados/default.html?nc=624"
    
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    #"download.default_directory": "C:/Users/DSTHREE/Documents/GITHUB/airflow/dags/utils",
})
    # Configuración para ingresar al explorador
    #driver = webdriver.Chrome(options = chrome_options)
    remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait= WebDriverWait(driver, 10)
    button = wait.until(EC.element_to_be_clickable((By.ID, "aCsv")))
    button.click()
    time.sleep(20)
    driver.quit()
    
    def data_transformation():
        df = pd.read_csv("Tabulado.csv")
        # Encuentra los índices donde comienzan los años en la columna 'Periodo'
        indices_anios = df[df['Periodo'].str.isnumeric()].index
        nuevo_df = df.T.iloc[1:].reset_index(drop=True)
        nuevo_df = nuevo_df.dropna(axis =1)
        num_columnas = len(nuevo_df.columns)
            
        years = range(2020,2025)
        # Lista de números romanos
        romanos = ['I', 'II', 'III', 'IV']
        # Crea una lista de nombres de columnas basados en el patrón
        nombres_columnas = [f"{anno}-{romano}" for anno in years for romano in romanos]
        nombres_columnas = nombres_columnas[:num_columnas]
        #Asigna los nombres de las columnas al nuevo DataFrame
        nuevo_df.columns = nombres_columnas 
        # Agrega una columna 'Estado' al nuevo DataFrame
        nuevo_df.insert(0, 'Estado', df.columns[1:])

        # Restablece el índice
        nuevo_df.reset_index(drop=True, inplace=True)
        nuevo_df.to_csv("Tabulado.csv", index = False)
    data_transformation()
INEGI_desempleo_scraper()

df_desempleo = pd.read_csv("Tabulado.csv")
df_shf = pd.read_csv("SHF_extract.csv")

def join_data(df_shf, df_desempleo):
    ## SHF data
    df_shf.Estado = df_shf.Estado.replace(r'\s+', '', regex=True)
    rename = {'BajaCalifornia':'Baja California', 'BajaCaliforniaSur':'Baja California Sur',
    'CiudaddeMéxico':'Ciudad de México', 'NuevoLeón':'Nuevo León', 'QuintanaRoo':'Quintana Roo',
    'SanLuisPotosí':'San Luis Potosí'
    }
    df_shf.Estado = df_shf.Estado.replace(rename)

    ## Desempleo data
    rename = {'Coahuila de Zaragoza':'Coahuila', 'Distrito Federal':'Ciudad de México',
    'Michoacán de Ocampo':'Michoacán', 'Veracruz de Ignacio de la Llave':'Veracruz'
    }
    df_desempleo.Estado = df_desempleo.Estado.replace(rename)

    ## Join data
    data = list()
    for estado in df_desempleo.Estado:
        for column in df_desempleo.columns:
            if column=="Estado":
                continue
            year_tri = column.split("-")
            shf = df_shf.loc[df_shf.Estado==estado][column].values[0]
            desempleo = df_desempleo.loc[df_desempleo.Estado==estado][column].values[0]
            data_ = {"Estado":estado, "Año":year_tri[0], "Trimestre":year_tri[1],
            "SHF":shf, "Desempleo":desempleo
            }
            data.append(data_)
    df_all = pd.DataFrame(data)
    return df_all
join_data(df_shf, df_desempleo)

URL actual https://www.gob.mx/cms/uploads/attachment/file/848146/_ndice_SHF_T2_2023.pdf


,Estado,Año,Trimestre,SHF,Desempleo
0,Aguascalientes,2020,I,122.5,3.350091
1,Aguascalientes,2020,II,123.9,4.533647
2,Aguascalientes,2020,III,125.4,5.192303
3,Aguascalientes,2020,IV,127.5,5.197538
4,Aguascalientes,2021,I,130.3,3.622400
...,...,...,...,...,...
443,Zacatecas,2022,II,136.3,3.298042
444,Zacatecas,2022,III,140.7,2.742564
445,Zacatecas,2022,IV,144.6,2.359975
446,Zacatecas,2023,I,148.4,3.369287
